# Предсказание размера зарплаты по объявлениям

In [1]:
import pandas

# Загрузка данных из файлов
data_train=pandas.read_csv('salary-train.csv')
# Тестовые данные не участвуют в обучении, в них нет целевой переменной
# Данные из реальной жизни, которые прогоняются через готовую модель для получения ответа
data_test=pandas.read_csv('salary-test-mini.csv')

def print_head():
    print '======= Salary Train ======='
    print data_train.head()
    print '======= Salary Test Mini ======='
    print data_test.head()
    
print_head()

======= Salary Train =======
                                     FullDescription LocationNormalized  \
0  International Sales Manager London ****k  ****...             London   
1  An ideal opportunity for an individual that ha...             London   
2  Online Content and Brand Manager// Luxury Reta...  South East London   
3  A great local marketleader is seeking a perman...            Dereham   
4  Registered Nurse / RGN  Nursing Home for Young...   Sutton Coldfield   

  ContractTime  SalaryNormalized  
0    permanent             33000  
1    permanent             50000  
2    permanent             40000  
3    permanent             22500  
4          NaN             20355  
======= Salary Test Mini =======
                                     FullDescription LocationNormalized  \
0  We currently have a vacancy for an HR Project ...      Milton Keynes   
1  A Web developer opportunity has arisen with an...         Manchester   

  ContractTime  SalaryNormalized  
0     contract  

In [2]:
def lower(text):
    return text.lower()

# Функция нормализации данных:
# 1) оставляем только буквы и цифры, все переводим в lowercase
# 2) заменяем пропуски на строку "nan"
def normalize_df(df):
    df['FullDescription'] = df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True).apply(lower)
    df['ContractTime'].fillna('nan', inplace = True)
    df['LocationNormalized'].fillna('nan', inplace = True)
    df['LocationNormalized'] = df['LocationNormalized'].replace('[^a-zA-Z0-9]', ' ', regex = True).apply(lower)

    return df

# Нормализуем оба набора данных
data_test=normalize_df(data_test)
data_train=normalize_df(data_train)

In [3]:
print_head()

======= Salary Train =======
                                     FullDescription LocationNormalized  \
0  international sales manager london     k      ...             london   
1  an ideal opportunity for an individual that ha...             london   
2  online content and brand manager   luxury reta...  south east london   
3  a great local marketleader is seeking a perman...            dereham   
4  registered nurse   rgn  nursing home for young...   sutton coldfield   

  ContractTime  SalaryNormalized  
0    permanent             33000  
1    permanent             50000  
2    permanent             40000  
3    permanent             22500  
4          nan             20355  
======= Salary Test Mini =======
                                     FullDescription LocationNormalized  \
0  we currently have a vacancy for an hr project ...      milton keynes   
1  a web developer opportunity has arisen with an...         manchester   

  ContractTime  SalaryNormalized  
0     contract  

In [4]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()

# Каждый город и тип работы будут отдельной колонкой
# Объявления в строчках, в соответствующих колонках для нужных городов и типов работы ставим 1
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

print '======= Salary Train ======='
print X_train_categ[:5]
print '======= Salary Test Mini ======='
print X_test_categ

X_train_categ

======= Salary Train =======
  (0, 2)	1.0
  (0, 957)	1.0
  (1, 2)	1.0
  (1, 957)	1.0
  (2, 2)	1.0
  (2, 1392)	1.0
  (3, 2)	1.0
  (3, 471)	1.0
  (4, 1)	1.0
  (4, 1495)	1.0
======= Salary Test Mini =======
  (0, 0)	1.0
  (0, 1040)	1.0
  (1, 2)	1.0
  (1, 997)	1.0


<60000x1766 sparse matrix of type '<type 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vzr=TfidfVectorizer(min_df=5)
vzr

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [6]:
# Строим TF-IDF индекс -- для каждого слова оценивается его важность
# относительно всей коллекции "документов" (у нас документ - это текст объявления)
# Каждая колонка - отдельное слово 
# Пропускаем слова, если они встречаются менее чем в пяти разных документах (min_df=5)
# Ячейка не пустая если слово встречается в данном объявлении

mx_train=vzr.fit_transform(data_train['FullDescription'])

In [7]:
print mx_train[:5,0:700]

mx_train

  (0, 609)	0.0435026522837
  (0, 482)	0.0261783638125
  (0, 477)	0.0759214010654
  (0, 101)	0.0774013219018
  (0, 655)	0.0589658990281
  (1, 679)	0.169263206104
  (2, 678)	0.0317735376665
  (2, 482)	0.0215038677594
  (2, 655)	0.0242183756039
  (4, 61)	0.0881786407467
  (4, 488)	0.0575900138819
  (4, 141)	0.0897324297354
  (4, 201)	0.108872742103


<60000x22861 sparse matrix of type '<type 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [8]:
from scipy.sparse import hstack

# Объединяем таблички: добавляем колонки справа
X_union=hstack([mx_train, X_train_categ])
X_union

<60000x24627 sparse matrix of type '<type 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [9]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

from datetime import datetime

start=datetime.now()


# По полученным данным строим модель

# Линейная регрессия с L2-регуляризацией Тихонова по целевой переменной "зарплата"
# alpha соответствует коэффициенту регуляризации (штраф за сложность модели, препятствует переобучению)

ridge=Ridge(alpha=1)

# ridge=RidgeCV(scoring="r2", cv=2, store_cv_values=False)
ridge.fit(X_union,data_train['SalaryNormalized'])

end=datetime.now()
print end-start

0:00:05.087823


In [10]:
# Объединяем таблички с тестовыми данными
mx_test=vzr.transform(data_test['FullDescription'])
X_union_test=hstack([mx_test, X_test_categ])
X_union_test

<2x24627 sparse matrix of type '<type 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [11]:
import numpy as np

# Предсказываем значения зарплаты для тестового набора
np.round(ridge.predict(X_union_test), 2)

array([ 56588.1 ,  37207.51])